# Data Wrangling

In [ ]:
# %load utils/imports.py
import numpy as np
import pandas as pd

from utils import *
from utils.styles import *

%matplotlib inline

In [ ]:
# %load utils/plotting.py
import seaborn as sns

from plotly.offline import init_notebook_mode, iplot
import cufflinks as cf

init_notebook_mode()
cf.go_offline()

# 01b. From Messy to Tidy Data with Pandas

## Tidying messy datasets

#### What's considered tidy data?

- Each variable forms a column
- Each observation forms a row.
- Each type of observational unit forms a table.

### What are the five most common problems with messy data sets? How do we fix them?

#### Our messy data identification + cleaning process:

- Check if it satisfies the above criteria
- List your observations
- The problem
- CLEAN it!
- Check criteria again

#### Question

    What are the most common problems with messy data? Can you name them? (hidden - enter to reveal)

<!---
- Column headers are values, not variable names.
- Multiple variables stored in one column
- Variables are stored in both rows and columns
- Multiple types of observational units are stored in the same table.
- A single observational unit is stored in multiple tables.
--->

## #1 - PEW Demographics

This dataset explores the relationship between income and religion in the US. It comes from a report produced by the _Pew Research Center_, an American think-tank that collects data on attitudes to topics ranging from religion to the internet, and produces many reports that contain datasets in this format.

The dataset has three variables, **religion**, **income**, and **frequency**.

In [ ]:
fn = 'pew.csv'
download_data(fn)
pew = pd.read_csv('data/' + fn)
grid(pew)

### What's wrong with this dataset? Does it satisfy the aforementioned criteria?

- Is each column a variable?
- Is each row an observation?
- Does each observational unit (a row) form a table?

### Observation:
- **Religion** is a variable.
- Each category of the **income** variable is presented in six separate columns.
- **Frequency** is considered a value here.

## Messy Data #1: Column headers are values, not variable names.
### How to fix it: 
Reshape the table with three columns: religion, income, frequency.

In [ ]:
# reshape table with three columns (religion, income, frequency)
grid(pd.melt(pew, id_vars='religion',var_name='income',value_name='frequency'))

## Does this qualify as tidy data?

This form is **tidy** because **each column represents a variable** and **each row represents an observation**, in this case a demographic unit corresponding to a combination of religion and income.

## #2 - WHO Tuberculosis

This tuberculosis dataset comes from the World Health Organisation, and records the counts of confirmed tuberculosis cases by country, year, and demographic group. The demographic groups are broken down by sex (m, f) and age (0-14, 15-24, 25-34, 35-44, 45-54, 55-64, unknown).

In [ ]:
fn = 'tb.csv'
download_data(fn)
tb = pd.read_csv('data/' + fn)
grid(tb)

### What's wrong with this dataset? Does it satisfy the aforementioned criteria?

- Is each column a variable?
- Is each row an observation?
- Does each observational unit (a row) form a table?

### Observation:
- There are a lot of missing values
- The m04...fu columns represent **two** variables: sex and age

## Messy data #2 - Multiple variables stored in one column
### How to fix it:
- First we gather up the non-variable columns:
- Column headers in this format are often separated by a non-alphanumeric character (e.g. ., -, \_, :), or have a fixed width format, like in this dataset. `str.split()` makes it easy to split a compound variables into individual variables. You can either pass it a regular expression to split on (the default is to split on non-alphanumeric columns), or a vector of character positions. In this case we want to split after the first character:

In [ ]:
# It has 22 columns, two which are actually labels, and 20 which contain values
tb.shape

In [ ]:
#  reshape table with three columns (iso2, year, m04..fu, n)
tb = pd.melt(tb, id_vars=['iso2','year'],var_name='sex_age',value_name='frequency')

In [ ]:
#  drop missing values, 
tb.dropna(subset=['frequency'], inplace=True)

In [ ]:
#  split m04...fu field into two variables = sex, age
tb[['sex','age']] = tb.sex_age.str.extract('([mf])([\d\w]+)',expand=True)

In [ ]:
# drop the original duplicated column
tb = tb[['iso2','year','sex','age','frequency']]
grid(tb)

## Does this qualify as tidy data?
Storing the values in this form resolves a problem in the original data. We want to compare rates, not counts, which means we need to know the population. In the original format, there is no easy way to add a population variable. It has to be stored in a separate table, which makes it hard to correctly match populations to counts. In tidy form, adding variables for population and rate is easy because they’re just additional columns.

## #3 - Global Historical Climatology Network Weather

The code below loads daily weather data from the Global Historical Climatology Network for one weather station (MX17004) in 
Mexico for five months in 2010.

In [ ]:
fn = 'weather.csv'
download_data(fn)
weather = pd.read_csv('data/' + fn)
grid(weather)

### What's wrong with this dataset? Does it satisfy the aforementioned criteria?

- Is each column a variable?
- Is each row an observation?
- Does each observational unit (a row) form a table?

### Observation:
- id, year, month are variables in individual columns. That is fine.
- But there are variables spread across columns (i.e. day) as well as variables across rows (i.e. tmax and tmin)

    Answer (hidden)

<!-- Variables are stored in both rows and columns -->

## Messy data #3 - Variables are stored in both rows and columns
### How to fix it:

- We first gather the day columns:
- We also drop the missing values, making them implicit rather than explicit. This is ok because we know how many days are in each month and can easily reconstruct the explicit missing values. 
- This dataset is mostly tidy, but the element column is not a variable; it stores the names of variables. (Not shown in this example are the other meteorological variables prcp (precipitation) and snow (snowfall)). Fixing this requires the spread operation. This performs the inverse of gathering by spreading the element and value columns back out into the columns:

In [ ]:
# make *day* a variable
w = pd.melt(weather, id_vars=weather.columns[:4].tolist(),var_name='day')
# drop the leading 'd' in the value
w.day = w.day.str.replace('d','')
# spread *element* and *value* columns into two variables: tmax and tmin
labels = w.columns[:5].tolist()
wd = w.groupby(labels).sum().unstack(level=3).swaplevel(axis=1).reset_index()
wd.columns = wd.columns.droplevel(1)
# drop missing values
grid(wd.dropna().head())

## Does this qualify as tidy data?
This form is tidy: there’s one variable in each column, and each row represents one day.

## #4  - BIllboard Rankings

The Billboard dataset shown below records the date a song first entered the billboard top 100. It has variables for artist, track, date.entered, rank and week. The rank in each week after it enters the top 100 is recorded in 75 columns, wk1 to wk75. This form of storage is not tidy, but it is useful for data entry. It reduces duplication since otherwise each song in each week would need its own row, and song metadata like title and artist would need to be repeated. 

In [ ]:
billboard = pd.read_csv("https://raw.githubusercontent.com/hadley/tidy-data/master/data/billboard.csv")
grid(billboard)

### What's wrong with this dataset? Does it satisfy the aforementioned criteria?

- Is each column a variable?
- Is each row an observation?
- Does each observational unit (a row) form a table?

### Observation:
- There is duplication of facts about the song: artist, year, and time are repeated many times. (because one artist can have many top hits)
- This results in two types of observational units / records: One stores the song metadata (i.e. artist, year, name, time, genre). The other type is its ranking in the charts.

    Answer (hidden)

<!---
Multiple types of observational unit in one table? 
The two types: The song and its rank in each week
--->

## Messy data #4 - Multiple types of observational unit in one table

Each observational unit should be stored in its own table. This is closely related to the idea of database normalisation, where each fact is expressed in only one place. It’s important because otherwise inconsistencies can arise.

### How to fix it:

First, tidy the data.

We first need to tidy up by making **week** and **rank** variables / columns. Feel free to drop any missing values as they represent weeks that the song wasn't in the charts. Convert the week variable to an int, perhaps also figuring out the date corresponding to each week on the charts.

In [ ]:
# TODO make week and rank variables
# TIP : Select the columns you want to melt.

# TODO drop missing values

# TODO calculate the date from the week variable+year
# TIP : Are you familiar with pandas to_datetime and to_timedelta ? 

    Answer (hidden)

<!--- 
```python
# TODO make week and rank variables

# Helper function to gather the columns we want to melt
def gather( df, key, value, cols):
    id_vars = [ col for col in df.columns if col not in cols ]
    id_values = cols
    var_name = key
    value_name = value
    return pd.melt( df, id_vars, id_values, var_name, value_name )

# Select the columns to melt - the ones containing '.week'
cols = billboard.columns[billboard.columns.str.contains('.week')].tolist()
# Melt the week columns
bb = gather(billboard,'week','rank',cols)
# Clean up Week
bb.week = bb.week.str.extract('(\d)').astype(int)

# TODO drop missing values
bb = bb.dropna()

# TODO calculate the date from the week variable+year
bb.date = pd.to_datetime(bb['date.entered']) + pd.to_timedelta(bb.week, unit='w')

```
--->

Second, we need to break this dataset into two pieces: a song dataset which stores artist, song name and time, and a ranking dataset which gives the rank of the song in each week. 

- We first extract a song dataset:
- Then use that to make a rank dataset by replacing repeated song facts with a pointer to song details (a unique song id):

In [ ]:
# TODO extract song dataset with columns: artist, track, year, time. And give it a UID (song_id)
# TIP : Use the index of grouped objects as the UID

# TODO extra rank dataset with columnrs song_id date, week, rank. Set up a pointer to song details.
# Merge datasets in a selective way

    Answer (hidden)

<!--- 
```python
# TODO extract song dataset with columns: artist, track, year, time. And give it a UID (song_id)

# Sort values for clarity
bb = bb.sort_values(['artist.inverted','track','week'])
# User index of grouped items as UID
songs = bb.groupby(['artist.inverted','track'],as_index=False).head(1).reset_index().ix[:,:6]
# Clean up columns
song_cols = "song_id year artist track time genre".split()
songs.columns = song_cols

# TODO extra rank dataset with columnrs song_id date, week, rank. Set up a pointer to song details.

# Merge song_id back in, but only keep selected columns
rank_cols = "song_id date week rank".split(' ')
rankings = bb.merge(songs)[rank_cols]

# Results
songs.head(10)
# rankings.head(10)
```
--->

### Question: What kind of data structure is this?

Answer (hidden)
<!--- relational data model --->

Normalisation is useful for tidying and eliminating inconsistencies. However, there are few data analysis tools that work directly with relational data, so analysis usually also requires denormalisation or the merging the datasets back into one table.

## #5 - Babynames

It’s also common to find data values about a single type of observational unit spread out over multiple tables or files. 
These tables and files are often split up by another variable, so that each represents a single year, person, or location. 

e.g. Take 129 yearly baby name tables provided by the US Social Security Administration and combine them into a single table/file.

## Messy data #5 - A single observational unit is stored in multiple tables.

### How to fix it:
 As long as the format for individual records is consistent, this is an easy problem to fix:

- Read the files into a list of tables.

- For each table, add a new column that records the original file name (the file name is often the value of an important variable).

- Combine all tables into a single table.

### Resources

https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html

http://www.prometheusresearch.com/good-data-management-practices-for-data-analysis-tidy-data-part-2/